<a href="https://colab.research.google.com/github/asrulharunismail/BA-Machine-Learning/blob/main/multi_modal_vans_drones_map.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import folium
import pandas as pd
import requests
from branca.element import Template, MacroElement

def decode_polyline(polyline_str):
    coordinates = []
    index, lat, lng = 0, 0, 0
    while index < len(polyline_str):
        shift, result = 0, 0
        while True:
            byte = ord(polyline_str[index]) - 63
            index += 1
            result |= (byte & 0x1f) << shift
            shift += 5
            if not byte >= 0x20:
                break
        lat += ~(result >> 1) if result & 1 else result >> 1
        shift, result = 0, 0
        while True:
            byte = ord(polyline_str[index]) - 63
            index += 1
            result |= (byte & 0x1f) << shift
            shift += 5
            if not byte >= 0x20:
                break
        lng += ~(result >> 1) if result & 1 else result >> 1
        coordinates.append((lat / 1e5, lng / 1e5))
    return coordinates

def get_route(coord1, coord2):
    url = f'http://router.project-osrm.org/route/v1/driving/{coord1[1]},{coord1[0]};{coord2[1]},{coord2[0]}?overview=full'
    response = requests.get(url)
    if response.status_code != 200:
        return None
    return decode_polyline(response.json()['routes'][0]['geometry'])

def create_2Modal_map(nodes_drone, nodes_van, line_weight=2.5):
    map_center = [sum([node[0] for node in nodes_van[0]]) / len(nodes_van[0]),
                  sum([node[1] for node in nodes_van[0]]) / len(nodes_van[0])]
    map_object = folium.Map(location=map_center, zoom_start=10, tiles='OpenStreetMap')

    def add_route_to_map(route, colour, map_obj, use_osrm=True, place_markers=False):
        for i in range(len(route)):
            if place_markers or i == 0 or i == len(route) - 1:
                folium.CircleMarker(
                    location=route[i],
                    radius=4,
                    color=colour,
                    fill=True,
                    fill_color='rgba(0, 255, 0, 0)'
                ).add_to(map_obj)

            if i < len(route) - 1:
                if use_osrm:
                    route_path = get_route(route[i], route[i + 1])
                else:
                    route_path = [route[i], route[i + 1]]
                if route_path:
                    folium.PolyLine(route_path, color=colour, weight=line_weight, opacity=0.75).add_to(map_obj)

    for route in nodes_van:
        add_route_to_map(route, 'red', map_object, True, place_markers=True)

    for pair in nodes_drone:
        add_route_to_map(pair, 'blue', map_object, False, place_markers=True)

    return map_object

In [2]:
nodes_drone_example = [[[50.93302, -1.4351], [50.7691, -1.54548], [50.72243, -1.58952], [50.87604, -1.32719],[50.93302, -1.4351] ]]

nodes_van_example = [[[50.7691, -1.54548], [50.7691, -1.54548]],
 [[50.93302, -1.4351],[50.92137, -1.45597],[50.85182, -1.41617],[50.81895, -1.37344],[50.87001, -1.40338],[50.93302, -1.4351]],
 [[50.93302, -1.4351],[50.89783, -1.37279],[50.89631, -1.36949],[50.89729, -1.37284],[50.91469, -1.33261],[50.91435, -1.27512],[50.93302, -1.4351]],
 [[50.93302, -1.4351],[50.81471, -1.57666],[50.7564, -1.65799],[50.93302, -1.4351]],
 [[50.93302, -1.4351],[50.9906, -1.49382],[50.87228, -1.57661],[50.9194, -1.4899],[50.93302, -1.4351]],
 [[50.93302, -1.4351],[50.91941, -1.41975],[50.9158, -1.41568],[50.91338, -1.42157],[50.93297, -1.35229],[50.9393, -1.37854],[50.9341, -1.39571],[50.97561, -1.43692],[50.93302, -1.4351]]]

In [5]:
map_2M = create_2Modal_map(nodes_drone_example, nodes_van_example, line_weight=3)
map_2M.save('2Modal_map.html')

map_2M